### Author: Jose Miguel Bautista
### Updated: 05/31/2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import heapq
from scipy.sparse import csr_matrix 
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse.csgraph import shortest_path
from numba import njit

In [2]:
# Skiena graph from Figure 6.3 
# Used in all subsequent cells for consistency, but you may swap out with arbitrary square matrices for experimentation. 
test = np.array([
    [0, 5, 7, 9, 0, 0, 0], 
    [0, 0, 0, 7, 12, 0, 0],
    [0, 0, 0, 4, 0, 2, 5],
    [0, 0, 0, 0, 4, 3, 0],
    [0, 0, 0, 0, 0, 7, 0],
    [0, 0, 0, 0, 0, 0, 2],
    [0, 0, 0, 0, 0, 0, 0],
], dtype=float)
test = (test+test.T)

# Depending on the code used, either 0 or infinity is more conveninent for marking non-connection. 
# By default I will use infinity, but swap as needed. 
test[test == 0] = np.inf 
np.fill_diagonal(test, 0) # necessary for shortest paths, not strictly for MST

# Weighted Graphs

Our work up to this point has been on simple, unweighted graphs.  
Such graphs can be represented mathematically with boolean adjacency matrices.  
Weighted graphs can be represented by the same system, except instead of boolean entries, we log the weights in a *distance matrix*.  
Such weights can convey more information about the relationship (e.g. correlation strength, physical distance, etc.).  
This construction also requires us to rethink some of the algorithms and potential problems.  

# Minimum Spanning Trees

Recall that a tree is a connected graph without cycles.  
A *spanning tree* is a subgraph of G with all the same vertices of $G$ (but not the same edges), and is itself a tree.  
A *minimum spanning tree (MST)* is the spanning tree of weighted graph $G$ s.t. sum of edges is minimized.  
**Note:** The MST is not necessarily unique, such as if the graphs has multiple edges of the same weight.  

This has practical applications to network design.  
Historically, MSTs were applied to electrical grids in Moravia (Czech republic), by [Boruvka](https://en.wikipedia.org/wiki/Bor%C5%AFvka%27s_algorithm).  
You clearly want to have a grid that is connected, in order to maintain a proper service.  
But wiring all of them together takes time, effort, and literal miles of wiring.  
Given a set of power stations, how do you connect them in a way that requires the least amount of wire?    

Boruvka managed to do this in the 60s, without the aid of computers.  
What this means for us is that we're *not* going to go over it because it's more complicated (for now).  
Once we get through the relatively simpler algorithms, you can go look at his work in your own time.  


## Prim's Algorithm

Prim's algorithm is one of the simplest methods to make an MST.  

**Pseudocode:**
1. Pick a starting vertex (arbitrary).
1. While there are unvisited vertices:
    1. Greedily select the lowest weight edge between a tree and nontree vertex. 
    1. Add the edge and corresponding vertex to the current tree.

### Proof of Correctness
Clearly Prim's algorithm creates a spanning tree because 
1. It only adds edges from tree to nontree vertices, and
1. It adds until all vertices are included.
Is it minimal?	
	
>**Theorem: Correctness of Prim's Algorithm**  
Given a graph $G$ with only positive edges, Prim's algorithm correctly identifies its minimum spanning tree.  
**Proof by Induction:**  
First consider the case of the subgraph consisting of 2 vertices sharing an edge.  
Prim's algorithm has no choice but to select the lone edge, which will be an MST, so the base case is shown.  
Now for the inductive step, assume Prim's algorthm has already constructed an MST on a subgraph.  
In the next step, Prim's algorithm will select the minimum edge to an unvisited vertex.  
Because there are only positive edges, the net path distance is strictly increasing in path length.  
Consequently, the shortest paths between disjoint sets of vertices have to be of path length 1.  
The resulting tree must therefore be an MST on the (larger) subgraph.  
Thus, Prim's algorithm produces a correct MST.  
**QED**

I will clarify that Skiena's proof by contradiction uses the same argument, where selected edges cannot be incorrect because the edges are positive.  
This leads to a more general idea of why a greedy algorithm may/may not fail.  
Greedy algorithms generally fail because they get "stuck" in locally optimal solutions.  
For MSTs on graphs with positive weights, the algorithm can't get stuck because no negative edges (that lower net path distance) exist. 

### Performance
How fast Prim's algorithm runs depends on the implementation/data structures.  
Below I have some demo code that takes a graph as an adjacency matrix (dense), and does a brute force check for the next edge to add.  
This should remind you of the selection sort algorithm, and the way it is sped up by using a better data structure.  
In fact, the fastest way to do this is also using heaps, which I demo below as well.  
**Exercise:** Figure out the time complexities, both from analysis of the code, and numerically.  

In [3]:
# Note: has no guardrails for disconnected/negative graphs. 

def prim(adj_matrix, start_vertex):
    #print("Starting vertex: ", start_vertex)
    v_num = len(adj_matrix)
    in_tree = [start_vertex] 
    edge_list = []
    net_weight = 0
    
    while len(in_tree)< v_num:
        best_weight = np.inf
        best_edge = None
        for i in in_tree:
            for j in range(v_num):
                weight = adj_matrix[i, j]
                if (j not in in_tree)&(weight < best_weight):
                    best_weight = weight 
                    best_edge = (i, j, weight)
                    
        v1, v2, weight = best_edge
        net_weight += best_weight
        in_tree.append(v2)
        edge_list.append(best_edge)
        
    return edge_list, net_weight
    
test[test == 0] = np.inf
np.fill_diagonal(test, 0)

%timeit outEdge, outWeight = prim(test, 0)
prim(test, 0)

227 µs ± 5.91 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


([(0, 1, 5.0),
  (0, 2, 7.0),
  (2, 5, 2.0),
  (5, 6, 2.0),
  (5, 3, 3.0),
  (3, 4, 4.0)],
 23.0)

In [4]:
# Improved version that takes csr format and uses heaps to organize edge exploration
# Rather than rechecking for connecting edges every time, it just logs them into a heap
# Note: still has no guardrails

def prim_csr(data, indices, indptr, start_vertex):
    #print("Starting vertex: ", start_vertex)
    
    v_num = len(indptr)-1
    in_tree = [start_vertex]
    visited = [False] * v_num
    visited[start_vertex] = True
    edge_heap = []
    edge_list = []
    net_weight = 0

    def add_edges(u):
        for i in range(indptr[u], indptr[u + 1]):
            v = indices[i]
            w = data[i]
            if not visited[v]:
                heapq.heappush(edge_heap, (w, u, v))

    add_edges(start_vertex)

    while (len(in_tree) < v_num):
        w, u, v = heapq.heappop(edge_heap)
        if visited[v]:
            continue
        visited[v] = True
        in_tree.append(v)
        edge_list.append((u, v, w))
        net_weight += w
        add_edges(v)

    return edge_list, net_weight

test[np.isinf(test)] = 0 # note: csr_matrix only recognizes 0s for non-connection
test_csr = csr_matrix(test)
data = test_csr.data
indices = test_csr.indices  
indptr = test_csr.indptr
  
%timeit prim_csr(data, indices, indptr, 0)
prim_csr(data, indices, indptr, 0)

16.4 µs ± 162 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


([(0, 1, 5.0),
  (0, 2, 7.0),
  (2, 5, 2.0),
  (5, 6, 2.0),
  (5, 3, 3.0),
  (3, 4, 4.0)],
 23.0)

## Kruskal's Algorithm

Kruskal's algorithm is an alternative to Prim's algorithm.  
It is actually similar to the heap-variant of Prim, except now we heap from the get-go.  
Then we add edges, forming (at least initially) disconnected trees.  
We keep adding edges as long as the edge does not connect a tree to itself. 

**Pseudocode:**
1. Put all the edges in a priority queue by weight.
1. While the queue is non-empty:
    1. Test if the edge connects 2 vertices in the same tree
    1. Add the edge if it does not, reject if it does.

### Exercise - Proof of Correctness
Clearly Kruskal's algorithm creates a spanning tree because 
1. It only adds tree edges, explicitly avoiding back/cross edges
1. All reachable vertices have to be added (a lone vertex is always a disconnected tree)

I will now ask you to prove that such a spanning tree is minimal.  
See Skiena p. 197 if you really need the answer, but I insist you work it out in the same way the proof of Prim's algorithm.  
    
### Performance
How fast Kruskal's algorithm runs depends (again) on implementation and choice of data structures.  
You could code up your own version, and you should at some point to get the order of addition right.  
But I will at this point note that scipy's `minimum_spanning_tree` does use Kruskal's algorithm.  
Below I have demo code for it.  

What you should notice is that it is faster than our naive Prim, but slower than our heaped Prim.  
**Exercise:** Why is it slower? Be precise in your statements. 


In [5]:
%timeit minimum_spanning_tree(test_csr)
minimum_spanning_tree(test_csr).toarray().astype(int)

107 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


array([[0, 5, 7, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0],
       [0, 0, 0, 0, 4, 3, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2],
       [0, 0, 0, 0, 0, 0, 0]])

## Union-find
The slow part of Kruskal’s algorithm in the book (initially) is the connectivity check, which is done by a breadth-first search.   
This is overkill, because we only need to test if 2 vertices to be connected are in same tree + merge, not a full search.  
These can be implemented by union and find operations on sets, specifically data structures called [disjoint sets](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.DisjointSet.html), where 
- `FIND(i)` – Return the root of tree containing element i
- `UNION(i,j)` – Link the root of one of the tree (i) to the root of the other (j).  

Internally, such structures are disconnected trees where the pointers run from child to parent.  
By following the pointer from any descendent node, `FIND` returns the "representative" of its set (the root node).  
Thus we only merge if the representatives of 2 sets are different.  
If we want union and find to be fast, it is best to maintain trees of low height.  
When merging trees $T_1$ and $T_2$ into new tree $T_3$, which one becomes the root? 
- Shorter tree: `HEIGHT`($T_3$) = `MAX`( `HEIGHT`($T_1$), `HEIGHT`($T_2$) ) $+ 1$
- Taller tree: `HEIGHT`($T_3$) = `MAX`( `HEIGHT`($T_1$), `HEIGHT`($T_2$) )  

Ergo, we should keep the taller tree as the root.  
Scipy uses a union-find structure in its implementation, but it also has 2 other techniques to speed things up.  
These are mentioned in the source code, and explained in Cormen 19, so I will go over them. 

### Union-by-Rank
Remember that we want to maintain trees of low height to keep the operations on the union-find as fast as possible.  
So typically some space is allocated to log the number of descendants per tree node for fast comparison.  
But note that the height of the tree grows (by 1) if and only if the input trees are the same height.  
So it is more actually efficient to store *ranks* rather than full sizes.  

A rank is initialized at 0 for all trees (lone vertices).  
When two trees of the same height are connected by an edge, either may become the parent of the other.  
In that case, the parent tree has its rank increased by 1, and the child's rank never changes beyond that point.  
As such, rank is an upper bound of tree height.  

### Path Compression

Remember that our only goal is to find the representative for checking connectivity.  
We want to use union-find over BFS because we don't need to search every path.  

But for that matter, we don't need to copy the actual graph structure into our union-find.  
In other words, we don't care about parents and grandparents in the actual graph, we just need to know the representative of the tree.  
We may as well make the representative the parent of all descendants in our union-find structure, which always has height 1. 

Again, the height only increases if merging trees of the same height, after which one may compress the tree.  
It should be clear that this compression can be done recursively (if the node is not the root, recursively check its parent).  
This pairs nicely with the union-by-rank method above, because rank is also decoupled from direct tree structure.  
Combining the two yields a theoretical time complexity that is nearly linear.  
It actually goes as an [inverse Ackermann function](https://en.wikipedia.org/wiki/Ackermann_function#Inverse), $\alpha(n)$, which is effectively a constant for reasonable input.  
See Cormen 19.4 or Tarjan’s work for the full details.  


# Shortest Paths

A path is a sequence of edges connecting two vertices.  
Finding the shortest path between vertices is a common problem:
- Google Maps - what is the fastest route between 2 locations.
- IP Routing - same as mapping
- Video games - how a character move to get somewhere when there are obstacles
- Linguistics - [how does one learn complex words?](https://pubmed.ncbi.nlm.nih.gov/39465458/)
- Bioinformatics - [how are different gene sets related?](https://pmc.ncbi.nlm.nih.gov/articles/PMC5726383/)

On unweighted graphs, we can do this in $O(n+m)$ using BFS.  
Weighted graphs have some complications.  
The weight of a path between two vertices is the sum of the weights of the edges on a path.  
But now the contributions of edges are no longer uniform.  
BFS will not work here because now, a sum of mutliple edges can be less than a single direct (but heavy) edge. 

Cycles with net negative weights render a "shortest path" meaningless.  
If a negative cycle exists, we can always travel around it more to arbitrarily reduce the weight of final path.  
This has applications to triangular arbitrage, but makes our immediate goal more complicated.  
Thus (initially) we will assume that all edge weights are positive.  
**Note:** Our minimum spanning tree algorithms are unaffected by negative edges.

In [scipy](https://github.com/scipy/scipy/blob/main/scipy/sparse/csgraph/_shortest_path.pyx), there are different algorithms to compute the shortest path, and we will briefly go over most of them.  
Below I have some demo code for the supported algorithms, though scipy can automatically choose which one to use based on what it thinks will be fastest.  
I will note that at the time of writing, its implementation of Dijkstra and Johnson's algorithms do not work if the associated matrices are not symmetric.  
So you will need to make your own versions for those if you need an asymmetric (read: directed) graph.  




In [6]:
start = 0
test[np.isinf(test)] = 0

print("Dijkstra's Algorithm \n")
dist_matrix, predecessors = shortest_path(csgraph=test, directed=False, indices=start, return_predecessors=True, method = 'D')
print("Distance from vertex/es ({}): ".format(start), dist_matrix)
print("Predecessor on shortest path: ", predecessors) #-9999 indicates no path
%timeit dist_matrix, predecessors = shortest_path(csgraph=test, directed=False, indices=start, return_predecessors=True, method = 'D')


Dijkstra's Algorithm 

Distance from vertex/es (0):  [ 0.  5.  7.  9. 13.  9. 11.]
Predecessor on shortest path:  [-9999     0     0     0     3     2     5]
616 µs ± 64.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
#start = 0 # must input full graph
test[np.isinf(test)] = 0

print("Floyd-Warshall Algorithm \n")
dist_matrix, predecessors = shortest_path(csgraph=test, directed=False, return_predecessors=True, method = 'FW')
print("Distance from vertexes: \n", dist_matrix)
print("Predecessor on shortest path: \n", predecessors) #-9999 indicates no path
%timeit dist_matrix, predecessors = shortest_path(csgraph=test, directed=False, return_predecessors=True, method = 'FW')


Floyd-Warshall Algorithm 

Distance from vertexes: 
 [[ 0.  5.  7.  9. 13.  9. 11.]
 [ 5.  0. 11.  7. 11. 10. 12.]
 [ 7. 11.  0.  4.  8.  2.  4.]
 [ 9.  7.  4.  0.  4.  3.  5.]
 [13. 11.  8.  4.  0.  7.  9.]
 [ 9. 10.  2.  3.  7.  0.  2.]
 [11. 12.  4.  5.  9.  2.  0.]]
Predecessor on shortest path: 
 [[-9999     0     0     0     3     2     5]
 [    1 -9999     3     1     3     3     5]
 [    2     3 -9999     2     3     2     5]
 [    3     3     3 -9999     3     3     5]
 [    3     3     3     4 -9999     4     5]
 [    2     3     5     5     5 -9999     5]
 [    2     3     5     5     5     6 -9999]]
291 µs ± 8.09 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
start = 0
test[np.isinf(test)] = 0

print("Bellman-Ford Algorithm \n")
dist_matrix, predecessors = shortest_path(csgraph=test, directed=False, indices=start, return_predecessors=True, method = 'BF')
print("Distance from vertex/es ({}): ".format(start), dist_matrix)
print("Predecessor on shortest path: ", predecessors) #-9999 indicates no path
%timeit dist_matrix, predecessors = shortest_path(csgraph=test, directed=False, indices=start, return_predecessors=True, method = 'BF')


Bellman-Ford Algorithm 

Distance from vertex/es (0):  [ 0.  5.  7.  9. 13.  9. 11.]
Predecessor on shortest path:  [-9999     0     0     0     3     2     5]
419 µs ± 26.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
start = 0
test[np.isinf(test)] = 0

print("Johnson's Algorithm \n") # skipped in discussion
dist_matrix, predecessors = shortest_path(csgraph=test, directed=False, indices=start, return_predecessors=True, method = 'J')
print("Distance from vertex/es ({}): ".format(start), dist_matrix)
print("Predecessor on shortest path: ", predecessors) #-9999 indicates no path
%timeit dist_matrix, predecessors = shortest_path(csgraph=test, directed=False, indices=start, return_predecessors=True, method = 'J')


Johnson's Algorithm 

Distance from vertex/es (0):  [ 0.  5.  7.  9. 13.  9. 11.]
Predecessor on shortest path:  [-9999     0     0     0     3     2     5]
603 µs ± 13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
start = 0
test[np.isinf(test)] = 0

print("Auto Select\n")
dist_matrix, predecessors = shortest_path(csgraph=test, directed=False, indices=start, return_predecessors=True, method = 'auto')
print("Distance from vertex/es ({}): ".format(start), dist_matrix)
print("Predecessor on shortest path: ", predecessors) #-9999 indicates no path
%timeit dist_matrix, predecessors = shortest_path(csgraph=test, directed=False, indices=start, return_predecessors=True, method = 'auto')


Auto Select

Distance from vertex/es (0):  [ 0.  5.  7.  9. 13.  9. 11.]
Predecessor on shortest path:  [-9999     0     0     0     3     2     5]
590 µs ± 15.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Dijkstra's Algorithm

The basic method for shortest paths is Dijkstra's algorithm, and it stems from the following observation:  
>Suppose a path $(s, \ldots, x, \ldots, t)$ is the shortest path from $s$ to $t$.  
Then the subpath $(s, \ldots, x)$ must be the shortest path from $s$ to $x$ on the way to $t$.   
If it were not, we could take the actual shortest path to $x$, then use the other half of the original path $(x, \ldots, t)$.

This suggests a *dynamic programming* approach.  
Dijkstra's algorithm proceeds in a series of rounds, each finding the shortest path from source $s$ to new vertex $x$.   
More distant vertices will have paths constructed from closer ones.   

**Pseudocode:**  
1. Initialize distance to source $s$.  
This is $0$ for $s$ itself, infinity for every other vertex.  
1. Set the unvisited vertex with smallest edge as `LAST` (initialized as $s$).  
1. From `LAST`, calculate distances to connected vertices going through `LAST`.  
1. For connected vertices, compare new distances to the current, and assign the smaller value.  
1. Mark `LAST` as visited.  
1. Repeat from step 2 until target vertex found.  

You might notice a similarity to Prim’s algorithm (both are greedy).  
Per iteration, you add exactly one vertex to the tree of best solutions in order of increasing cost.  
The difference is how they prefer/rate new vertices.  
- For Prim/MST, we only care about the (marginal) weight being added by potential edges. 
- For Kruskal/shortest path, we care about the overall path distance to source.

As a note on implementation, you can make your own version of Dijkstra's algorithm with priority, possibly following [this version](https://www.geeksforgeeks.org/dijkstras-shortest-path-algorithm-greedy-algo-7/).  
But in the scipy implementation, they use a special data structure called [Fibonacci heaps](https://en.wikipedia.org/wiki/Fibonacci_heap) to improve their asymptotic run times.   
We are not going to go over them, because it would take a while.  
But the basic idea is to use mutliple min-heaps (instead of just one) and handle them lazily, merging as needed.  
Effectively, the height of any heap in a Fibonacci heap is (on average) smaller than a full priority queue which should speed it up (but see the appendix). 
  
## Floyd-Warshall Algorithm and Distance Matrices

Dijkstra’s algorithm finds the shortest path between a pair of vertices, or more geenrally from any given source.  
Sometimes we need the shortest path between all pairs of vertices, such as if we want to characterize the "size" of the graph.  
We could run Dijkstra’s algorithm for all pairs, but we may be able to do better with math. 

### Distance Matrices And Distance Products
Similar to the adjacency matrix, for (weighted) graph $G(V, E)$, we can construct the distance matrix $D$:
\begin{align}
D[i, j] = 0 \,\,\, \quad \quad &\text{IF} \,\, i = j\\
D[i, j] = \infty \quad \quad &\text{IF} \,\, (i \neq j)\text{ AND }(v_i, v_j) \in E\\
D[i, j] = w(i, j)\,\, &\text{IF} \,\, (v_i, v_j) \in E
\end{align}
where $w(i, j)$ is the weight of the edge going from vertex $i$ to vertex $j$.  
This alone tells us the shortest direct path (no intermediate vertices).  
Unlike the adjacency matrix, powers of this matrix under [matrix multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication) are not that useful.  

Instead, the relevant operation you want to reach for is [min-plus multiplication](https://en.wikipedia.org/wiki/Min-plus_matrix_multiplication).  
This has some fancy names associated with it like "tropical semi-rings" but you don't need to worry about them.  
All you need to know is how to calculate it, which is functionally the same as the usual matrix product (read: a collection of dot products).  
The only differences are:
1. Replace all addition operations with `MIN` comparisons, and
2. Replace all multiplications with addition.

### Floyd-Warshall (FW)
Say we want to get from vertex $i$ to $j$, and assume we need to pass (at least) vertex $k$ on the best path.  
Key idea: treat all possible vertices $k$ as an intermediate node sequentially.  
When considering vertex $k$, vertices $0$ to $k-1$ will have already been processed.  
As before, shortest paths from vertices $0$ to $k-1$ will build the shortest paths to vertex $k$.   

**Pseudocode:**  
1. Initialize $V\times V$ distance matrix $D$. 
    1. Set all diagonal elements to $0$. 
    1. Assign appropriate weights to relevant matrix positions. 
    1. Set all other entries to infinity. 
1. For all possible intermediate vertices $k$:
    1. For all pairs of starting and ending vertices $(i, j)$ excluding k:
        1. Add distance $i\rightarrow k$ to distance $k \rightarrow j$. 
        1. If this value is lower than current distance $i\rightarrow j$, replace it. 

Nominally, FW is not faster than just running Dijkstra’s algorithm for every pair.  
If anything, it should be slower because Dijkstra's coukld be sped up with special data structures but this is necessarily O(V^3)$.  
As you see in the demo above, it can (counterintuitively) be faster.  
A further benefit is that it turns out FW can handle negative edges, as long as no negative cycles are present  
Dijkstra is not designed to handle any negative edges at all.  
In fact, FW can tell you if negative cycles are present because they will show up as negative diagonals.  

## Bellman-Ford (BF) Algorithm 

Dijkstra’s algorithm fails for negative edges because it never revisits nodes.  
With the distance matrix formalism, FW algorithm can overcome this.  
Bellman-Ford is a mix, in that it uses distance prioduct, but it only finds distances to one source rather than a full matrix.  
Like Dijkstra's algorithm, BF relaxes solutions, but relaxes all edges (not just outgoing ones).

**Pseudocode:**
1. Initialize a distance array ($0$ for itself, else infinity), denoted $d$.
1. For all edges $(u, v)$:
    1. Consider distance $d[u]$ of vertex $u$ to source $s$, and add any potential edges $w(u,v)$.
    1. Compare to all current distances $d[v]$ of vertex $v$ to source $s$.  
       If any potential edges make the source distance smaller, update $d[v]$.  
1. Repeat step 2, $V-1$ times.

You should note that this will make the corresponding row of the distance matrix from Floyd-Warshall.  
This is for a good reason: both FW and BF calculate a min-plus product.  
Whereas FW does it between a (distance) matrix and itself, BF does it between the matrix and a vector (a row of the distance matrix itself). 


# Appendix: Distance Product

By default, Python has no min-plus products, and unfortunately neither do scipy or numpy.  
So if we want to use it to calculate shortest paths (of arbitrary max path length), we need to make it ourselves.  
In the interest of encouraging you to look at stackexchange and stackoverflow more, and for instructional purposes, here are [some](https://stackoverflow.com/questions/47359743/how-to-make-min-plus-matrix-multiplication-in-python-faster) of the solutions I got there.  

As a brief description of each approach:
1. The first version is a basic implementation using column-wise loops. 
1. The second version improves the first by removing explicit loops via reshaping and broadcasting. 
1. The third version is similar to the second, though the reshaping uses `newaxis` instead of `expanded_dims`, as it is evidently much faster.  
I assume this is due to lack of function overhead and possibly cache locality, though I am not entirely certain.  

To verify that they work, I calculate to the $(V-1)$-th power and compare to the output from scipy's Floyd-Warshall.  
The actual times should vary depending on your systems, and if you use different inputs.  
On my setup, compared to the first version, there is about a factor of $\approx$ 2 and $\approx$ 5 for the second and third versions respectively.   
**Moral of the Story:** broadcasting is awesome, avoid loops where possible.  

In [11]:
# Reference distance matrix from Floyd-Warshall. 
test[test == 0] = np.inf
np.fill_diagonal(test, 0)
FW_dist_matrix, predecessors = shortest_path(csgraph=test, directed=False, return_predecessors=True, method = 'FW')
print(FW_dist_matrix)

[[ 0.  5.  7.  9. 13.  9. 11.]
 [ 5.  0. 11.  7. 11. 10. 12.]
 [ 7. 11.  0.  4.  8.  2.  4.]
 [ 9.  7.  4.  0.  4.  3.  5.]
 [13. 11.  8.  4.  0.  7.  9.]
 [ 9. 10.  2.  3.  7.  0.  2.]
 [11. 12.  4.  5.  9.  2.  0.]]


In [12]:
def min_plus_1(A, B):
    B = np.transpose(B)
    Y = np.zeros((len(B),len(A)))
    for i in range(len(B)):
         Y[i] = (A + B[i]).min(1)
    return np.transpose(Y)

test[test == 0] = np.inf
np.fill_diagonal(test, 0)
out = test

#%timeit min_plus_1(test, test)
#min_plus_1(test, test)
for i in range(len(test)):
    out = min_plus_1(out, test)

np.testing.assert_array_equal(out, FW_dist_matrix) # if successful, returns nothing

In [13]:
%%timeit
out = test
for i in range(len(test)):
    out = min_plus_1(out, test)


247 µs ± 2.45 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
def min_plus_2(A, B):
    return (np.expand_dims(A, 0) + np.expand_dims(B.T, 1)).min(axis=2).T

test[test == 0] = np.inf
np.fill_diagonal(test, 0)
out = test

#%timeit min_plus_2(test, test)
#min_plus_2(test, test)
for i in range(len(test)):
    out = min_plus_2(out, test)
    
np.testing.assert_array_equal(out, FW_dist_matrix) # if successful, returns nothing

In [15]:
%%timeit
out = test
for i in range(len(test)):
    out = min_plus_2(out, test)


101 µs ± 766 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [16]:
def min_plus_3(A, B):
    A = A[:, :, np.newaxis] # shape (a, b, 1)
    B = B[np.newaxis, :, :] # shape(1, b, c)
    
    # A+B will have shape (a, b, c)
    return (A + B).min(axis=1) 

test[test == 0] = np.inf
np.fill_diagonal(test, 0)
out = test

#%timeit min_plus_3(test, test)
#min_plus_3(test, test)
for i in range(len(test)):
    out = min_plus_3(out, test)
    
np.testing.assert_array_equal(out, FW_dist_matrix) # if successful, returns nothing   

In [17]:
%%timeit
out = test
for i in range(len(test)):
    out = min_plus_3(out, test)


55.4 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Appendix: Practical Performance / Compilers

You may be wondering how the distance products I calculated are (on my end) running faster than the scipy implementation of FW .  
For that matter, you may wonder why FW is faster than any of the single-pair algorithms.  
I would first note that I have only shown the time for fixed input, on a relatively small (dense) graph.  
This is both for clarity of explanation, and because I want you to check the time complexities of your own voilition.  
If you start changing the input (either the number of vertices or the relative number of edges), there is no guarantee these rankings will stay the same.  

Aside from that, I will also mention that scipy's Dijkstra uses Fibonacci heaps because they give the best *theoretical* performance.  
The *actual* performance ultimately depends on many things including cache locality and compiler optimizations.  
So the implementation may be the best for asymptotically large graphs.  
But we never said how large you would have to go for that to actually matter.  
In the end, you should test your work for a given system and problem.  
To give you a taste of this, I will mention one other concept: compilers.  

Recall that Python is a high-level language that is meant to be interpretable to humans.  
In order to have the computer execute it, the high level code must be translated to machine-readable code.  
For Python in most operations, it actually goes through a few translations (bytecode, PVM, C, etc.) before reaching that state.   
If we really want to speed up operations, we need to get to low-level instructions as fast as possible.   

The most direct way would be to learn lower-level languages (C++, Fortran, assembly, etc.), but clearly I'm not going to do that right now.  
Instead, we want to use a compiler that takes Python code and compiles more directly to machine-readable code.  
This is where things like [numba](https://numba.pydata.org/) and [JAX](https://docs.jax.dev/en/latest/quickstart.html) come in.  
They do exactly what we want: take at least some Python and numpy code, and use a (JIT) compiler to speed up machine translation and optimize execution. 
**Sidenote:** In case you were curuious, numba does its compilation through [LLVM](https://en.wikipedia.org/wiki/LLVM).

Below I put some demo code for a min-plus product calculator, and its numba-enahanced version.  
The base function uses a triply-nested `for`-loop to run element-by-element, so it is incredibly slow.  
But we need to cast it into this form because numba does not accept many of the functions of numpy that we used above, specifically the `min` portions.  
To use it with numba, I put the `@njit` decorator on the function.  

The times will depend on the system/inputs, but by default on my system(s), numba should have a factor of $\approx$ 100 speedup.  
In fact, the numba-enhanced version of (somewhat inelegant) code even beats the best broadcast-based version above by a factor of $\approx$ 5.  
Again, this does not account for the time complexity so the results are not generalizeable to larger graphs.  
Still, that is a huge improvement and we should pay more attention to opportunities like this.  
**Moral of the Story:** Learn low-level languages; failing that, for high-level languages, check if a more direct compiler exists/helps.  

In [18]:
def min_plus_4(A, B):
    n, m = A.shape
    m2, p = B.shape

    C = np.full((n, p), np.inf)
    for i in range(n):
        for j in range(p):
            for k in range(m):
                temp = A[i, k] + B[k, j]
                if temp < C[i, j]:
                    C[i, j] = temp
    return C

test[test == 0] = np.inf
np.fill_diagonal(test, 0)
out = test

#%timeit min_plus_4(test, test)
#min_plus_4(test, test)
for i in range(len(test)):
    out = min_plus_1(out, test)

np.testing.assert_array_equal(out, FW_dist_matrix) # if successful, returns nothing

In [19]:
%%timeit
out = test
for i in range(len(test)):
    out = min_plus_4(out, test)


1.3 ms ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
@njit
def njit_min_plus_4(A, B):
    n, m = A.shape
    m2, p = B.shape

    C = np.full((n, p), np.inf)
    for i in range(n):
        for j in range(p):
            for k in range(m):
                temp = A[i, k] + B[k, j]
                if temp < C[i, j]:
                    C[i, j] = temp
    return C

test[test == 0] = np.inf
np.fill_diagonal(test, 0)
out = test

#%timeit njit_min_plus_4(test, test)
#njit_min_plus_4(test, test)
for i in range(len(test)):
    out = njit_min_plus_4(out, test)

In [21]:
%%timeit
out = test
for i in range(len(test)):
    out = njit_min_plus_4(out, test)


11.4 µs ± 1.62 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
